# **관광지명은 각자 최종적으로 찾는 것으로 넣으면 다 나옵니다 !**
## 에러가 있는 경우 다른 리스트에 넣어 확인하는 방법으로 했습니다.

In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
# 예외처리
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException, NoSuchWindowException, WebDriverException

import requests
import re
import time
import pandas as pd
from bs4 import BeautifulSoup

# 사용할 csv 불러오기
* 파일 path 각자에 맞게 설정해줘야함.

In [48]:
# 각자 맞게 파일 path 설정
file = './review_tag.csv'

In [49]:
# csv 파일 읽기
raw = pd.read_csv(file)
df = raw.copy()

In [43]:
df[df.addr.isnull()].iloc[:, 5:7].drop_duplicates()
# df[df.addr.isnull()].iloc[:, 5:7].value_counts()

,attraction,location
0,서울어린이대공원,서울 광진구
18,섬세이 테라리움,서울 성동구
30,목인박물관 목석원,서울 종로구
43,용마폭포공원,서울 중랑구
55,국립민속박물관&국립민속박물관 어린이박물관,서울 종로구
...,...,...
2257,프레젠트모먼트,서울 마포구
2273,헌인릉 생태경관보전지역(오리나무림),서울 서초구
2284,은평한옥마을,서울 은평구
2304,우리유황온천,서울 광진구


1. 네이버 모바일에서 검색하기
2. 상단의 **지도** 탭 클릭하기
3. 검색한 관광지명에 맞는 구이름 찾고 해당 관광지 선택하기
4. **리뷰** 탭 클릭하기
5. **이런 점이 좋았어요** 찾고 저장하기
    * **이런 점이 좋았어요**가 있다면
        * **좋은 점** 목록이 있는지 확인하기
            1) **공개됩니다**라는 글자가 있다면 목록이 공개되지 않아 내용을 가져올 수 없음.
                * 관광지명과 해당 주소 list 저장. 리뷰내용, 선택한 사람수, 총 사람 수는 None으로 list 저장
            2) 목록이 있다면 내용 저장.
                * 관광지명, 주소, 리뷰내용, 선택한 사람 수, 총 사람 수
    * 없다면
        * 관광지명, 주소 저장. 리뷰내용, 선택 사람 수, 총 사람 수는 None

# 네이버 리뷰 크롤링 함수

## 지도 탭 선택하기 함수

In [24]:
# 지도 선택하기 함수
def click_지도():
    tabs = driver.find_elements(By.CSS_SELECTOR, 'a.tab')
    for i, tab in enumerate(tabs):
        try:
            if '지도' in tab.text:
                css = f'#_sch_tab > div > div:nth-child({i+1}) > a'
                button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css)))
                print(f'{i+1}번째에 지도 탭있음.')
                break
            else:
                continue
        except:
            print('지도 탭 없음.')
            
    return button

## 관광지 선택하기 함수
* 여기에서 **관광지 주소**를 **location**이라는 변수에 저장함.

In [26]:
# 지도에서 원하는 관광지 선택하기 함수
def click_관광지(x):
    # 해당 관광지의 구이름
    loca_short = x
    # 원하는 관광지명의 시군구명이 같은 항목 선택하기
    # <a href="#" class="item_address _btnAddress" data-nclicks="plc_ntl.roadon" data-cid="11796387" data-rank="1">서울특별시 광진구 능동로 216</a>
    addrs = driver.find_elements(By.CLASS_NAME, 'item_address._btnAddress')
    # 반복문 돌려서 원하는 구 이름 찾으면 선택하기
    for i, addr in enumerate(addrs):
        try:
            # 구이름이랑 맞으면
            if loca_short in addr.text:
                # 도로명주소 가져오기
                location = addr.text.split('\n')[1]
                print(location)
                # 찾고 있는 관광지 클릭 준비
                loc_css = f'#ct > div.search_listview._content._ctList > ul > li:nth-child({i+1}) > div.item_info > a.a_item.a_item_distance._linkSiteview'
                # loc_xpath = f'//*[@id="ct"]/div[2]/ul/li[{i+1}]/div[1]/a[2]'
                # //*[@id="ct"]/div[2]/ul/li[2]/div[1]/a[2]
                # #ct > div.search_listview._content._ctAddress > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview
                button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, loc_css)))
                print(f'{i+1}번째에 해당 관광지 있음.')
                break
            else:
                continue
        except:
            print('해당 관광지 없음.')
            
    return button, location   

## 리뷰 탭 선택하기 함수

In [25]:
# 카테고리 추출 후, 리뷰 탭 선택하기 함수
def click_리뷰():
    # 카테고리 선택
    # <span class="lnJFt">카페,디저트</span>
    category = driver.find_element(By.CLASS_NAME, 'lnJFt').text
    # <a href="/place/11796387/review" role="tab" class="tpj9w _tab-menu" aria-selected="false" title="" id="" style="width: 158px;"><span class="veBoZ">리뷰</span></a>
    tabs = driver.find_elements(By.CSS_SELECTOR, 'a.tpj9w._tab-menu > span.veBoZ')
    for i, tab in enumerate(tabs):
        try:
            if '리뷰' in tab.text:
                # #app-root > div > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-child(5)
                css = f'#app-root > div > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-child({i+1})'
                button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css)))
                print(f'{i+1}번째에 리뷰 탭있음.')
                break
            else:
                continue
        except:
            print('리뷰 탭 없음.')

    return button, category

## 이런 점이 좋았어요 찾고 저장하기 함수
* 여기에서 **주소**, **카테고리**, **총 참여 인원 수** 추출 가능

In [46]:
# '이런 점이 좋았어요' 찾고 저장하는 함수
def find_save_category(list_):
    # 이런 점이 좋았어요 확인
    good_point = driver.find_element(By.CLASS_NAME, 'place_section_header')
    if '이런 점이 좋았어요' in good_point.text:
        print('"이런 점이 좋았어요" 있음.')
                
        try:
            # <div class="zXXsw"><em>10</em>명 이상 참여하면<br>방문 키워드 통계가 공개됩니다!</div>
            blind = driver.find_element(By.CLASS_NAME, 'zXXsw')
            
            if '공개됩니다' in blind.text:
                print('리뷰수가 너무 적어 목록이 생성되지 않았습니다.')
                list_.append({'관광지명' : spot, '주소' : location, '목록' : category, '참여인원수' : None})
            
        except NoSuchElementException:
            
            # 참여인원수 추출
            # <span class="mholG">263명 참여</span>
            reviewer = driver.find_element(By.CLASS_NAME, 'mholG').text.split()[0]
            # 리스트에 관광지명, 도로명주소, 리뷰내용과 리뷰수를 딕셔너리로 담기
            list_.append({'관광지명' : spot, '주소' : location, '목록' : category, '참여인원수' : reviewer}) 
            # 저장 완료 표시                print(f'----- {n+1}번째 좋은 점 저장 -----')
        
    else:
        print('"이런 점이 좋았어요" 없음.')
        list_.append({'관광지명' : spot, '주소' : location, '목록' : category, '참여인원수' : None})

    return list_

# **리뷰 찾기 코드**

In [50]:
# 에러 안 난 결과 리스트
results = []
# 에러 난 결과 리스트
cant_find = []

# 사용할 DF
df = df[df.addr.isnull()].iloc[:, 5:7].drop_duplicates()

# 웹 드라이버 실행
driver = webdriver.Chrome()
# 웹 드라이버 최대화
driver.maximize_window()

# 반복문으로 관광지 돌리기
for i in range(df.shape[0]):
    try:
        # 관광지명 가져오기
        spot = df.iloc[i, 0]
        # 구 이름 가져오기
        loca_short = df.iloc[i, 1].split()[1]
        print('-'*20, i+1, ':', spot, '-'*20)
    
        # spot 찾기 
        driver.get(f"https://m.search.naver.com/search.naver?sm=mtp_sly.hst&where=m&query={spot}")
        time.sleep(6)
        
        # 지도 탭 찾기
        search_button = click_지도()
        if search_button != None:
            # 지도 선택하기    
            search_button.click()
            time.sleep(3)
            
            # 해당 관광지 찾기
            search_button, location = click_관광지(loca_short)
            if search_button != None:
                # 해당 관광지 선택하기
                search_button.click()
                time.sleep(3)
    
                # 리뷰 선택하기
                search_button, category = click_리뷰()
                if search_button != None:
                    # 리뷰 클릭
                    search_button.click()
                    time.sleep(3)
    
                    # 좋은점 확인하고 저장하기
                    results = find_save_category(results)
                    print(f'{spot} 검색 완료')
                    
                else:
                    print('"리뷰 탭"이 없어서 다음 관광지로 넘어갑니다.')
                    
            else:
                print('"해당 관광지"가 없어서 다음 관광지로 넘어갑니다.')
                
        else:
            print('"지도 탭"이 없어서 다음 관광지로 넘어갑니다.')

    except:
        print('오류로 실패 리스트에 인덱스로 저장합니다.')
        cant_find.append(i)

driver.quit()

-------------------- 1 : 서울어린이대공원 --------------------
10번째에 지도 탭있음.
서울특별시 광진구 능동로 216
1번째에 해당 관광지 있음.
3번째에 리뷰 탭있음.
"이런 점이 좋았어요" 있음.
서울어린이대공원 검색 완료
-------------------- 2 : 섬세이 테라리움 --------------------
10번째에 지도 탭있음.
서울특별시 성동구 서울숲2길 44-1 지하1층 섬세이 테라리움
1번째에 해당 관광지 있음.
4번째에 리뷰 탭있음.
"이런 점이 좋았어요" 있음.
섬세이 테라리움 검색 완료
-------------------- 3 : 목인박물관 목석원 --------------------
10번째에 지도 탭있음.
서울특별시 종로구 창의문로5길 46-1
1번째에 해당 관광지 있음.
3번째에 리뷰 탭있음.
"이런 점이 좋았어요" 있음.
목인박물관 목석원 검색 완료
-------------------- 4 : 용마폭포공원 --------------------
10번째에 지도 탭있음.
서울특별시 중랑구 용마산로 250-12 용마폭포공원관리사무소
1번째에 해당 관광지 있음.
2번째에 리뷰 탭있음.
"이런 점이 좋았어요" 있음.
용마폭포공원 검색 완료
-------------------- 5 : 국립민속박물관&국립민속박물관 어린이박물관 --------------------
10번째에 지도 탭있음.
서울특별시 종로구 삼청로 37
1번째에 해당 관광지 있음.
4번째에 리뷰 탭있음.
"이런 점이 좋았어요" 있음.
국립민속박물관&국립민속박물관 어린이박물관 검색 완료
-------------------- 6 : 서울 운현궁 --------------------
10번째에 지도 탭있음.
서울특별시 종로구 삼일대로 464 운현궁
1번째에 해당 관광지 있음.
3번째에 리뷰 탭있음.
"이런 점이 좋았어요" 있음.
서울 운현궁 검색 완료
-------------------- 7 : 여의도샛강생태공원 --------------

## 실행결과 만들어진 list 확인
* results, cant_find

In [51]:
# 잘 뽑힌 데이터
results

[{'관광지명': '서울어린이대공원',
  '주소': '서울특별시 광진구 능동로 216',
  '목록': '테마파크',
  '참여인원수': '2,699명'},
 {'관광지명': '섬세이 테라리움',
  '주소': '서울특별시 성동구 서울숲2길 44-1 지하1층 섬세이 테라리움',
  '목록': '전시관',
  '참여인원수': '152명'},
 {'관광지명': '목인박물관 목석원',
  '주소': '서울특별시 종로구 창의문로5길 46-1',
  '목록': '박물관',
  '참여인원수': '135명'},
 {'관광지명': '용마폭포공원',
  '주소': '서울특별시 중랑구 용마산로 250-12 용마폭포공원관리사무소',
  '목록': '도시,테마공원',
  '참여인원수': '64명'},
 {'관광지명': '국립민속박물관&국립민속박물관 어린이박물관',
  '주소': '서울특별시 종로구 삼청로 37',
  '목록': '박물관',
  '참여인원수': '28명'},
 {'관광지명': '서울 운현궁',
  '주소': '서울특별시 종로구 삼일대로 464 운현궁',
  '목록': '궁궐',
  '참여인원수': '47명'},
 {'관광지명': '여의도샛강생태공원',
  '주소': '서울특별시 영등포구 여의도동 49',
  '목록': '자연,생태공원',
  '참여인원수': '18명'},
 {'관광지명': '난지한강공원',
  '주소': '서울특별시 마포구 한강난지로 162 한강공원 난지안내센터',
  '목록': '시민공원',
  '참여인원수': '105명'},
 {'관광지명': '국립항공박물관', '주소': '서울특별시 강서구 하늘길 177', '목록': '박물관', '참여인원수': '789명'},
 {'관광지명': '경동시장', '주소': '서울특별시 동대문구 고산자로36길 3', '목록': '시장', '참여인원수': '49명'},
 {'관광지명': '용산가족공원',
  '주소': '서울특별시 용산구 서빙고로 137 국립중앙박물관',
  '목록': '근린공원',
  '참여인원수'

In [52]:
# 직접 찾아야 하는 데이터
cant_find

[114]

# DF 만들기

In [57]:
# 함수로 만들기
def makingDF(result, error, df):
    # 데이터 프레임 만들어주기
    results_raw = pd.DataFrame(result)
    results_df = results_raw.copy()
    
    # 에러난 데이터만 있는
    if error != []:
        error = df.iloc[error, :].reset_index(drop=True)

    return results_df, error

In [58]:
# 함수 적용
success_df, error_df = makingDF(results, cant_find, df)

## 실행결과 만들어진 DF 확인

In [60]:
# 확인
# success_df
# error_df

,attraction,location
0,필동면옥,서울 중구
